# **Installing packages and importing libraries**

In [ ]:

!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd

!apt-get install -y -qq glpk-utils

!apt-get install -y -qq coinor-cbc



     |████████████████████████████████| 9.4MB 3.1MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 256kB 43.6MB/s 
     |████████████████████████████████| 163kB 42.9MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-


# **Ex1: Part 1,2,3** - *Problem Fomulation, Model, and Solver*

In [ ]:
coef = np.loadtxt('lab6_practice_coef.txt', delimiter=',')

# create a model
model_lab6 = ConcreteModel()

N = coef.shape[1]-1 #we declare a Python variable N denoting the number of decision variables  


M = coef.shape[0]-1 #we declare a Python variable M denoting the number of constraints

# coefficients for objective function can be obtained by accessing the coef array
obj_coef = coef[0,:-1]

# We can segregate the coefficients for constraints into a separate array
constr_coef = coef[1:,:-1]

# We can segregate the RHS of constraints into a separate array
constr_rhs = coef[1:,-1]


# set of column indices for original decision variables x: since we have N/2 variables, we can take column indices to be the list [0,1,2,...,N-1]
col_indices_x = np.arange(int(N/2))


#declare the original decision variables in the model
model_lab6.x = Var(col_indices_x)

col_indices_u = np.arange(int(N/2))


#declare the new decision variables in the model
model_lab6.u = Var(col_indices_u, domain=NonNegativeReals)

# set of row indices: since we have M contraints, we can take row indices to be the list [0,1,2,...,M-1]
row_indices = np.arange(M)

# create a ConstraintList to hold multiple constraints
model_lab6.constraints = ConstraintList()

for i in row_indices:
  model_lab6.constraints.add(summation(constr_coef[i][0:int(N/2)],model_lab6.x) + summation(constr_coef[i][int(N/2):N],model_lab6.u)  >= constr_rhs[i])

# add the model objective
model_lab6.objective = Objective(expr = summation(obj_coef[0:int(N/2)],model_lab6.x) + summation(obj_coef[int(N/2):N],model_lab6.u), sense=minimize)  

model_lab6.pprint()

cbc_solver = SolverFactory('cbc')
result = cbc_solver.solve(model_lab6)

print('\n')
print('Status :',result.solver.status)
print('Termination Condition :',result.solver.termination_condition)
print('The optimal vlaue of the objective function is :',model_lab6.objective())
print('\n')
print('The value of the decision variables are :')
for i in range(int(N/2)):
  print('[x',i,'] : ',model_lab6.x[i].value)
  
for i in range(int(N/2)):
  print('[u',i,'] : ',model_lab6.u[i].value)

print('Constraints ')

model_lab6.constraints.display()



3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    u_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

2 Var Declarations
    u : Size=5, Index=u_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
    x : Siz

***Constraints that are active : 1,2,4,5,8,10,12,14,15,16***

# **Ex1: Part 4 to 11** - *Problem Fomulation, Model, and Solver for Option 2*

We will now consider the absolute value function appearing in the objective function. Consider the optimization problem $\textbf{(OP)}$:
\begin{align}
\min \ |x_1| + |x_2| + |x_3| + |x_4| + |x_5| &  \nonumber \\
{\rm{s.t.}} \ 85 x_1 + 92 x_2 + 45 x_3 + 27 x_4 + 31 x_5 & \geq 1 \nonumber \\
92 x_1 + 54 x_2 + 22 x_3 + 20 x_4 + 7 x_5 & \geq 1 \nonumber \\
96 x_1 + 67 x_2 + 29 x_3 + 20 x_4 + 11 x_5 & \geq 1 \nonumber \\
-91 x_1 - 57 x_2 - 33 x_3 - 23 x_4 - 12 x_5 & \geq 1 \nonumber \\
-99 x_1 - 75 x_2 - 26 x_3 - 24 x_4 - 41 x_5 & \geq 1 \nonumber \\
-98 x_1 - 99 x_2 - 57 x_3 - 45 x_4 - 65 x_5 & \geq 1 \nonumber
\end{align}


In this approach we will use some fundamental properties of real numbers. Indeed, for any real number $x$, we can write $x=a-b$ where $a\geq 0, b\geq 0$. 

Hence we can substitute $|x| = |a-b|, \ a\geq 0, b\geq 0$. 

Now we will consider another interesting property of real numbers: 
$|u-v| = u+v \iff u\geq 0, v\geq 0, uv = 0$. 

Using this property we can write $|x| = a+b, \ a\geq 0, \ b\geq 0$. Note that this replacement will imply that $ab=0$. 

Thus we can transform the optimization problem $\textbf{(OP)}$ into a new optimization problem $\textbf{(OP2)}$ where $x_i = a_i - b_i, \text{ and } |x_i| = a_i + b_i, \ \forall i \in \{1,\ldots,5\}$ where $a_i \geq 0, b_i \geq 0, \forall i \in \{1,\ldots,5\}$. Thus we have the new optimization problem $\textbf{(OP2)}$: 

\begin{align}
\min \ (a_1 + b_1) +  (a_2 + b_2) + (a_3 + b_3) + (a_4 + b_4) + (a_5 + b_5) &  \nonumber \\
{\rm{s.t.}} \ 85 (a_1-b_1) + 92 (a_2-b_2) + 45 (a_3-b_3) + 27 (a_4-b_4) + 31 (a_5-b_5) & \geq 1 \nonumber \\
92 (a_1-b_1) + 54 (a_2-b_2) + 22 (a_3-b_3) + 20 (a_4-b_4) + 7 (a_5-b_5) & \geq 1 \nonumber \\
96 (a_1-b_1) + 67 (a_2-b_2) + 29 (a_3-b_3) + 20 (a_4-b_4) + 11 (a_5-b_5) & \geq 1 \nonumber \\
-91 (a_1-b_1) - 57 (a_2-b_2) - 33 (a_3-b_3) - 23 (a_4-b_4) - 12 (a_5-b_5) & \geq 1 \nonumber \\
-99 (a_1-b_1) - 75 (a_2-b_2) - 26 (a_3-b_3) - 24 (a_4-b_4) - 41 (a_5-b_5) & \geq 1 \nonumber \\
-98 (a_1-b_1) - 99 (a_2-b_2) - 57 (a_3-b_3) - 45 (a_4-b_4) - 65 (a_5-b_5) & \geq 1 \nonumber \\ 
a_i \geq 0, \ b_i \geq 0, & \ \forall i \in \{1,\ldots,5\}. \nonumber 
\end{align}


In [ ]:
#Loading all the coefficients in the coef_2
coef_2 = np.loadtxt('lab6_practice_coef_OP2.txt', delimiter=',')

#Variable for the index of variables a and b
N2 = int((coef_2.shape[1]-1) /2)

#Creating separate arrays for coefs of a and b. Also creating an array for the rhs values
coef_a = coef_2[:,:N2]
coef_b = coef_2[:,N2:-1]
rhs = coef_2[:,-1]


variable_index = np.arange(int((coef_2.shape[1]-1) /2))

model = ConcreteModel()
model.a = Var(variable_index,domain= NonNegativeReals)
model.b = Var(variable_index,domain= NonNegativeReals)

model.obj = Objective(expr = summation(coef_a[0],model.a)+ summation(coef_b[0],model.b),sense=minimize )
model.cons = ConstraintList()

for i in range(coef_2.shape[0]-1):
  model.cons.add(summation(coef_a[i+1],model.a)+summation(coef_b[i+1],model.b) >= rhs[i+1]) 

for i in range(N2):
  model.a[i].setlb(0)
  model.b[i].setlb(0)

model.pprint()

result_2 = cbc_solver.solve(model)

print('\n')
print('Status :',result_2.solver.status)
print('Termination Condition :',result_2.solver.termination_condition)
print('The optimal vlaue of the objective function is :',model.obj())
print('\n')
print('The value of the decision variables are :')

for i in range(N2):
  print('[a',i,']:',model.a[i].value)

for i in range(N2):
  print('[b',i,']:',model.b[i].value)

print('\n')
print('Constraints are as follows : ')

model.cons.display()


3 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}

2 Var Declarations
    a : Size=5, Index=a_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
    b : Size=5, Index=b_index
        Key : Lower : Val

ACTIVE CONSTRAINTS : 1,2,4,5

# **Ex1: Part 12** - *Comparison*

In [ ]:

df = pd.DataFrame(columns=['x','u','a','b','a+b'])


df['x'] = np.array([model_lab6.x[i].value for i in range(N2)])
df['u'] = np.array([model_lab6.u[i].value for i in range(N2)]) 
df['a'] = np.array([model.a[i].value for i in range(N2)])
df['b'] = np.array([model.b[i].value for i in range(N2)])
df['a+b'] = np.array([(model.b[i].value +model.a[i].value) for i in range(N2)])

df

,x,u,a,b,a+b
0,-0.046449,0.046449,0.000000,0.046449,0.046449
1,0.186124,0.186124,0.186124,0.000000,0.186124
2,-0.171318,0.171318,0.000000,0.171318,0.171318
3,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.144065,0.144065,0.000000,0.144065,0.144065


***REMARKS***

We observe that ui is just the absolute value of the xi. and also that ui is the sum  ai+bi.

*We also expected this answer because in the second part of the question we changed the objective funtion by replacing the xi with ai+bi. And to minize the absolute value of xi, we used ui.*
